<a href="https://colab.research.google.com/github/BiancadeFrancisco/BigData_SparkSQL/blob/main/SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark #==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=7c812214fbbbe05c30a736b227ab7679f910504a1722a8ccce77753c6ff70951
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2023-08-22 14:25:37--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  56.0MB/s    in 0.2s    

2023-08-22 14:25:38 (56.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# ConfigureSparkUI
conf = SparkConf().set('spark.ui.port', '4050')
sc = SparkContext(conf=conf)
sc.stop()

spark = (
    SparkSession.builder                  # Método da classe que constrói a sessão spark
      .appName("Meu Primeiro App Spark")  # Nome do App Spark
      .getOrCreate())                     # Verifica se há uma sessão ativa, e se não há, cria uma nova sessão


In [ ]:
!curl -s http://localhost:4040/api/tunnels

# SparkSQL

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType

caminho_csv = "./base_de_dados.csv"

schema_base_pix = StructType([
    StructField('id', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('parte_debitada_nome', StringType()),
    StructField('parte_debitada_conta', StringType()),
    StructField('parte_debitada_banco', StringType()),
    StructField('parte_creditada_nome', StringType()),
    StructField('parte_creditada_conta', StringType()),
    StructField('parte_creditada_banco', StringType()),
    StructField('chave_pix_tipo', StringType()),
    StructField('chave_pix_valor', StringType()),
    StructField('data_transacao', TimestampType())
])

df = spark.read.csv(
    path=caminho_csv,
    header=True,
    sep=";",
    schema=schema_base_pix,
    timestampFormat="dd/MM/yyyy HH:mm"
)
spark.read.csv(
    path=caminho_csv,
    header=True,
    sep=";",
    schema=schema_base_pix,
    timestampFormat="dd/MM/yyyy HH:mm"
).createOrReplaceTempView("base_pix")   # CRIEI UMA TABELA COM O NOME BASE_PIX

In [ ]:
spark.sql("select * from base_pix limit 10").show()

+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+-------------------+
| id|   valor| parte_debitada_nome|parte_debitada_conta|parte_debitada_banco|parte_creditada_nome|parte_creditada_conta|parte_creditada_banco|chave_pix_tipo|chave_pix_valor|     data_transacao|
+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+-------------------+
|  1|    9.93|Dra. Ana Carolina...|            79470453|              Nubank|       Maysa da Cruz|             67162333|                 Itau|           cpf|     8439752610|2022-02-18 13:28:00|
|  2|   15.38|        Ana Caldeira|            19689668|                Itau|        Evelyn Sales|             60005091|             Bradesco|           cpf|    27145380617|2022-04-08 01:47:00|
|  3|   57.58|    Arthur Gonca

Porém, como saber se a manipulação de dados com Dataframes não é mais rápida que SQL?

Para isso vamos propor um group by das duas maneiras e verificar qual é o plano de execução que o spark cria.

In [ ]:
group_sql = spark.sql("select chave_pix_tipo, count(*) from base_pix group by chave_pix_tipo")

In [ ]:
group_dataframe = df.groupBy('chave_pix_tipo').count()

In [ ]:
print("SQL Group")
group_sql.explain()

print("DataFrame Group")
group_dataframe.explain()

SQL Group
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[chave_pix_tipo#74], functions=[count(1)])
   +- Exchange hashpartitioning(chave_pix_tipo#74, 200), ENSURE_REQUIREMENTS, [plan_id=41]
      +- HashAggregate(keys=[chave_pix_tipo#74], functions=[partial_count(1)])
         +- FileScan csv [chave_pix_tipo#74] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/base_de_dados.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<chave_pix_tipo:string>


DataFrame Group
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[chave_pix_tipo#52], functions=[count(1)])
   +- Exchange hashpartitioning(chave_pix_tipo#52, 200), ENSURE_REQUIREMENTS, [plan_id=54]
      +- HashAggregate(keys=[chave_pix_tipo#52], functions=[partial_count(1)])
         +- FileScan csv [chave_pix_tipo#52] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/bas

In [ ]:
spark.sql(
  """
    select chave_pix_tipo, sum(valor)
    from base_pix
    group by 1
  """
).show()

+--------------+------------------+
|chave_pix_tipo|        sum(valor)|
+--------------+------------------+
|       celular|         207778.46|
|         email|499009.38000000006|
|           cpf| 659513.3499999997|
+--------------+------------------+



In [ ]:
spark.sql(
  """
    select chave_pix_tipo, round(sum(valor), 2)
    from base_pix
    group by 1
  """
).show()

+--------------+--------------------+
|chave_pix_tipo|round(sum(valor), 2)|
+--------------+--------------------+
|       celular|           207778.46|
|         email|           499009.38|
|           cpf|           659513.35|
+--------------+--------------------+



In [ ]:
spark.sql(
  """
    select chave_pix_tipo, round(sum(valor), 2) as sum_valor
    from base_pix
    group by 1
  """
).show()

+--------------+---------+
|chave_pix_tipo|sum_valor|
+--------------+---------+
|       celular|207778.46|
|         email|499009.38|
|           cpf|659513.35|
+--------------+---------+



In [ ]:
spark.sql(
  """
    select chave_pix_tipo, count(*) as count
    from base_pix
    group by 1
  """
).show()

+--------------+-----+
|chave_pix_tipo|count|
+--------------+-----+
|       celular|   22|
|         email|   29|
|           cpf|   49|
+--------------+-----+



In [ ]:
spark.sql(
    """
      select parte_creditada_banco, valor,
        row_number() over (partition by parte_creditada_banco order by valor desc) as row_number
      from transacoes_pix
      limit 10
    """
).show()

AnalysisException: ignored

In [ ]:
spark.sql(
  """
  with base_pix_row_number as(
    select
      parte_creditada_banco,
      data_transacao,
      row_number() over (partition by parte_creditada_banco order by data_transacao desc) as row_number
    from base_pix
  ) select
      parte_creditada_banco,
      data_transacao
    from base_pix_row_number
    where row_number = 1
    order by data_transacao desc
  """
).show()

+---------------------+-------------------+
|parte_creditada_banco|     data_transacao|
+---------------------+-------------------+
|                 Itau|2022-12-15 01:29:00|
|                  BTG|2022-12-08 23:47:00|
|               Nubank|2022-11-19 19:25:00|
|             Bradesco|2022-08-07 17:01:00|
+---------------------+-------------------+



Porém, não precisa ficar limitado somente a execução de queries SQL.

Podemos pegar o resultado de uma query e retorná-la para um DataFrame!

In [ ]:
df_window = spark.sql(
  """
  with base_pix_row_number as(
    select
      parte_creditada_banco,
      data_transacao,
      row_number() over (partition by parte_creditada_banco order by data_transacao desc) as row_number
    from base_pix
  ) select
      parte_creditada_banco,
      data_transacao
    from base_pix_row_number
    where row_number = 1
    order by data_transacao desc
  """
).show()

+---------------------+-------------------+
|parte_creditada_banco|     data_transacao|
+---------------------+-------------------+
|                 Itau|2022-12-15 01:29:00|
|                  BTG|2022-12-08 23:47:00|
|               Nubank|2022-11-19 19:25:00|
|             Bradesco|2022-08-07 17:01:00|
+---------------------+-------------------+



In [ ]:
from pyspark.sql.functions import col

In [ ]:
df.selectExpr(
    "date(data_transacao) as date_data_transacao",
    "va"
).groupBy('date_data_transacao').count().orderBy(col('count').desc()).show()

AnalysisException: ignored

Exercício
1. Vimos que há dois dias em que houve duas transações pix. Descubra são os ids dessas transações.

2. Vimos que há dois dias em que houve duas transações pix. Descubra quais chaves pix foram utilizadas para realizar as transações.

In [ ]:
lista_datas = spark.sql(
  """
  select
    date(data_transacao) as date_data_transacao
  from base_pix
  group by 1
  having count(*) > 1
  """
).collect()[0][0]
lista_datas

datetime.date(2022, 2, 26)